In [1]:
from pathlib import Path
import pickle
from joblib import Parallel, delayed
from itertools import product
from smount_predictors.src.SeamountHelp import PipelinePredictor
from smount_predictors import SeamountHelp
import numpy as np
import pandas as pd
import simplekml
import xarray as xr

In [2]:
longitude_pairs = []
for lon in range(-180, 180, 20):
    longitude_pairs.append((lon, lon + 20))

latitude_pairs = []
for lat in range(-60, 90, 20):
    latitude_pairs.append((lat, lat + 20))


latlons = list(product(longitude_pairs, latitude_pairs))
latlons = latlons

In [3]:
model = pickle.load(open('out/cluster_tuned_model.pkl', 'rb'))
data_p = Path('data/vgg_swot_masked.grd')

In [4]:
redictions = pd.DataFrame(columns=['lon', 'lat', 'z', 'class', 'cluster'])
from interfaces_exclude.exclude_interface import exclude_interface
def predict_zone(zone):
    lon = zone[0]
    lat = zone[1]
    # print(f'Predicting zone {lon}-{lat}')
    data = SeamountHelp.readAndFilterGRD(data_p, lat, lon).to_dataframe().reset_index()
    # masked_data = exclude_interface(data).to_dataframe().reset_index()
    zone_pred = model.predict(data[['lon', 'lat', 'z']])
    return zone_pred

predictions = Parallel(n_jobs=-1)(delayed(predict_zone)(zone) for zone in latlons)
for idx, df in enumerate(predictions):  # ensure non-overlapping cluster numbers
    df['cluster'][df['cluster'] != -1] = df['cluster'][df['cluster'] != -1] + ((idx**2) * len(np.unique(df['cluster'])))
predictions = pd.concat(predictions)
predictions['lon'] = np.degrees(predictions['lon'])
predictions['lat'] = np.degrees(predictions['lat'])
global_predictions = xr.Dataset.from_dataframe(predictions.set_index(['lon', 'lat']))
global_predictions.to_netcdf('out/global_predictions.nc')

/Users/m1hawks/seamount_picking/code/smount_predictors/src/SeamountHelp.py:351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cluster'][data['class'] == 1] = self.clusterer.labels_
/Users/m1hawks/seamount_picking/code/smount_predictors/src/SeamountHelp.py:351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cluster'][data['class'] == 1] = self.clusterer.labels_
/Users/m1hawks/seamount_picking/code/smount_predictors/src/SeamountHelp.py:351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [5]:
mounts = predictions.groupby('cluster').mean().reset_index()
kml = simplekml.Kml()
for i, row in mounts.iterrows():
    kml.newpoint(name=f'{row.cluster}', coords=[(row.lon, row.lat, row.z)])
kml.save('out/global_mounts.kml')

In [6]:
! open out/global_mounts.kml

In [10]:
mounts.shape

(96517, 5)

In [11]:
pd.read_csv('data/all.xyhrdnc', sep=' ', header=None, names=['lon', 'lat', 'z', 'h', 'r', 'd', 'n', 'c'])

,lon,lat,z,h,r,d,n,c
0,-0.508333,-7.808333,2200,16.0116,-4759.604492,KW-00006,1,NaN
1,-1.858333,-7.825000,2100,15.2838,-4376.917480,KW-00016,1,NaN
2,-0.958333,-4.775000,1400,10.1892,-4516.997070,KW-00025,1,NaN
3,-4.625000,-4.675000,1600,11.6448,-4583.973633,KW-00043,1,NaN
4,-13.308333,-7.525000,2000,14.5560,-2967.866455,KW-00060,1,NaN
...,...,...,...,...,...,...,...,...
43964,-19.875000,-33.725000,1900,13.8282,-3602.549072,SIO2-08564,0,NaN
43965,-20.825000,-34.141667,1400,10.1892,-3774.492188,SIO2-08565,0,NaN
43966,-16.975000,-34.758333,1200,8.7336,-3354.338135,SIO2-08566,0,NaN
43967,-20.108333,-56.925000,1500,10.9170,-4634.202637,SIO2-08567,0,NaN
